In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob

In [ ]:
dir = '/content/drive/MyDrive/2022 빅데이터처리및응용 팀프로젝트'
files = glob.glob(os.path.join(dir, '*.csv'))

In [ ]:
type_columns = ['category_code_type1','category_code_type2','category_code_type3','category_code_type4']
col1 = ['0', '2', '3', '1']
col2 = ['0', '1', '3', '2']


def preprocessing(df):
    category_code = df.category_code.str.split('.')
    category_code = category_code.apply(lambda x:pd.Series(x))

    c_df = category_code

    na_type3 = c_df[c_df['2'].isna()]
    na_type4 = c_df[c_df['3'].isna()].drop(na_type3.index)

    all_type = c_df.drop(list(na_type3.index) + list(na_type4.index))
    all_type.columns = type_columns

    na_type3 = na_type3[col1]
    na_type4 = na_type4[col2]

    na_type3.columns = type_columns
    na_type4.columns = type_columns

    cate_df = pd.concat([all_type,na_type3,na_type4],axis=0)
    cate_df = cate_df.sort_index()

    new_df = df.drop('category_code',axis=1)
    new_df[type_columns] = cate_df

    valid_cols = new_df.drop(['event_time', 'product_id', 'category_id', 'user_id', 'user_session','price'],axis=1).columns

    new_df.reset_index(drop=True, inplace=True)


    category_manager = {}

    for col in valid_cols:
        category_manager[f'{col}_dict'] = dict()
        for i, v in enumerate(new_df[col].unique()):
            if pd.isna(v):
                category_manager[f'{col}_dict'][v] = 0
            else:
                category_manager[f'{col}_dict'][v] = i+1


    encoding_df = new_df.copy()

    for vcol, (k,v) in zip(valid_cols,category_manager.items()):
        encoding_df[vcol] = [v[val] for val in encoding_df[vcol]]

    return encoding_df

In [ ]:
for i, f in enumerate(files):
  df = pd.read_csv(f)
  df = df.dropna(axis = 0)
  result = preprocessing(df)
  if i ==0:
    out = result
  else:
    out = pd.concat([out, result], axis=0)

In [ ]:
out.to_csv('output.csv',index=False)

NameError: ignored